<font color = green >

# Text classification: topic modeling 

</font>

<font color = green >

### Latent Dirichlet allocation (LDA)

</font>

Typically used to detect underlying topics in the text documents

**Input** : text documents and number of topics 
<br>
**Output**: Distribution of topics for each document (that allows to assign th one with highest probability) and word distribution for each topic 

**Assumptions**:
- Documents with similar topics use similar groups of words 
- Documents are probability distribution over latent topics 
- Topics are probability distribution over words


<font color = green >

#### Generative process
<br>
</font>

LDA considers the every document is created the following way:

1) Define number if words in the document
<br>
2) Chose the topic mixture over the fixed set of topics (e.g. 20% of topic A, 30% of topic A, and 50% of topic A)
<br>
3) Generate the words by:
<br>
   -pick the topic based on document's multinomial distribution 
<br>
   -pick the word based on topic's multinomial distribution 

<img src = "img/topics_modeling.png" height=500 width= 800 align="left">



<font color = green >

#### Recall
</font>


#### Binomial distribution

$$p(k/n)\quad =\quad C^{ k }_{ n }\cdot p^{ k }(1-p)^{ n-k }\quad =\quad \frac { n! }{ k!(n-k)! } p^{ k }(1-p)^{ n-k }$$

Example: Probability of 6 of 10 for fear coin: 
$$p(6,4)\quad =\quad C^{ 6 }_{ 10 }\cdot {0.5}^{ 6 }(0.5)^{ 4 }\quad = 210 \cdot 0.015625 \cdot 0.0625 = 0.205078125$$


#### Multinomial distribution

$$p(n_{ 1 }n_{ 2 }...n_{ k })\quad =\quad \frac { n! }{ n_{ 1 }!n_{ 2 }!...n_{ k }! } p^{ n_{ 1 } }_{ 1 }p^{ n_{ 2 } }_{ 2 }...p^{ n_{ k } }_{ k }$$

Example (three outcomes): <br>
n = 12 (12 games are played),<br>
n1 = 7 (number won by Player A),<br>
n2 = 2 (number won by Player B),<br>
n3 = 3 (the number drawn),<br>
p1 = 0.4 (probability Player A wins)<br>
p2 = 0.35(probability Player B wins)<br>
p3 = 0.25(probability of a draw)<br>
$$p(7,2,3)\quad =\quad \frac {12!}{ 7! \cdot 2! \cdot3 ! }  \cdot 0.4^{7} \cdot 0.35^{2} \cdot0.25^{3} = 0.0248$$




<font color = green >

#### Maximul Likelihood Estimation

</font>

<br>

**Recall** 
<br> Known are text documents and number $K$ of topics 

**Target**:
<br>Within all possible topics distribution for all documemnts and all possible words distribution for topics, shoose the one wich maximizes probability of all text documents.

**Approach** :
<br>
1) Randomly assign each word of each document to $K$ topics 
<br>
2) Iterate the following process till convergence (steady assignments of w to topics) 
<br>$\quad\quad$For each document $d$: 
<br>
    $\quad\quad\bullet$ Assume that all topic assignment except current one are correct     
    $\quad\quad\bullet$ For each word $w$ in $d$:           
    $\quad\quad\quad$ - For every topic $t$ compare the the score for hypothesis that w is in this topic $t$:
   <br>$\quad\quad\quad\quad\quad score (t) =  p(t | d) \cdot p (w |t),$
   <br>$\quad\quad\quad\quad p(t|d)$ is proportion of all words in d from t,
    <br>$\quad\quad\quad\quad p(w|t)$ is share of word w in topic t.  
    $\quad\quad\quad$ - Assign the word w to the topic with max score
    <br>$\quad\quad\bullet$ Iterate through all $w$ in $d$:           
$\quad\quad$Iterate through all $d$

Te results is matrix of distribution of words in topics  
Note: The computed topics are just words distribution, i.e. need to summarize them somehow. 


<font color = green >

## Gensim LDA 

</font>



In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords

<font color = green >

### Define the text documents 

</font>



In [3]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]


<font color = green >

### Tokenize, clean, and stem

</font>



In [4]:
en_stop  = set(stopwords.words('english'))
p_stemmer = PorterStemmer()

def tokenize(doc_set):
    texts = []
    for doc in doc_set:
        # tokenize document string
        raw = doc.lower()
        tokens = word_tokenize(raw)

        # remove stop words from tokens
        tokens = [token for token in tokens if token not in en_stop]

        # stem tokens
        tokens = [p_stemmer.stem(token) for token in tokens]

        # add tokens to list
        texts.append(tokens)
    return texts

texts = tokenize(doc_set)
texts[0]

['brocolli',
 'good',
 'eat',
 '.',
 'brother',
 'like',
 'eat',
 'good',
 'brocolli',
 ',',
 'mother',
 '.']

<font color = green >

### Convert tokenized documents into a "id <-> term" dictionary

</font>



In [5]:
dictionary = corpora.Dictionary(texts) # this is alternative way - without using count vectorizer
print (type(dictionary), dictionary)
for k,w in dictionary.items():
    print (k,w)

<class 'gensim.corpora.dictionary.Dictionary'> Dictionary(34 unique tokens: [',', '.', 'brocolli', 'brother', 'eat']...)
0 ,
1 .
2 brocolli
3 brother
4 eat
5 good
6 like
7 mother
8 around
9 basebal
10 drive
11 lot
12 practic
13 spend
14 time
15 blood
16 caus
17 expert
18 health
19 increas
20 may
21 pressur
22 suggest
23 tension
24 better
25 feel
26 never
27 often
28 perform
29 school
30 seem
31 well
32 profession
33 say


<font color = green >

### Create gensim corpus

</font>



In [6]:
print ('\nconvert tokenized documents into a document-term matrix')
corpus = [dictionary.doc2bow(text) for text in texts] # id and count
for item in corpus:
    print (item)


convert tokenized documents into a document-term matrix
[(0, 1), (1, 2), (2, 2), (3, 1), (4, 2), (5, 2), (6, 1), (7, 1)]
[(1, 1), (3, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]
[(1, 1), (10, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]
[(0, 1), (1, 1), (3, 1), (7, 1), (10, 1), (21, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]
[(1, 1), (2, 1), (5, 1), (18, 2), (32, 1), (33, 1)]


<font color = green >

### Generate LDA model

</font>



In [7]:
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=2, id2word=dictionary, passes=20, random_state= 0)


<font color = green >

### Review topics 

</font>



In [8]:
ldamodel.print_topics(num_topics=2,num_words=10)

[(0,
  '0.098*"." + 0.076*"brocolli" + 0.076*"good" + 0.055*"mother" + 0.055*"brother" + 0.054*"health" + 0.054*"eat" + 0.033*"," + 0.033*"like" + 0.033*"spend"'),
 (1,
  '0.060*"drive" + 0.059*"pressur" + 0.059*"." + 0.036*"," + 0.036*"never" + 0.036*"often" + 0.036*"increas" + 0.036*"perform" + 0.036*"seem" + 0.036*"well"')]

<font color = green >

### Classify the new text 

</font>



In [9]:
test_doc_list = ["Some experts suggest that car may cause increased blood pressure. professionals say that brocolli is good "]
test_texts = tokenize(test_doc_list)
test_corpus = [dictionary.doc2bow(text) for text in test_texts ]
test_doc_topics = ldamodel.get_document_topics(test_corpus)
print ('\nget topics:')
for el in test_doc_topics: # loop over all tests in provided list
    print(el)


get topics:
[(0, 0.37417653), (1, 0.62582344)]


<font color = green >

### Sample of topic modeling on large dataset

</font>



In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle

<font color = green >

#### Load "News" data 

</font>



In [39]:
import os
cwd= os.getcwd()
path = os.path.join(cwd,)
fn=  os.path.join(path , 'newsgroups')

with open(fn, 'rb') as f:
    newsgroup_data = pickle.load(f)

<font color = green >

#### Review data

</font>



In [40]:
print (type(newsgroup_data))
print ('len of documents = {:,}\n'.format(len(newsgroup_data)))

newsgroup_data[0]

<class 'list'>
len of documents = 2,000



"The best group to keep you informed is the Crohn's and Colitis Foundation\nof America.  I do not know if the UK has a similar organization.  The\naddress of\nthe CCFA is \n\nCCFA\n444 Park Avenue South\n11th Floor\nNew York, NY  10016-7374\nUSA\n\nThey have a lot of information available and have a number of newsletters.\n \nGood Luck."

<font color = green >

#### Define custom vectorizer

</font>



In [41]:
three_words_pattern = r"\b\w{3,}\b"
vectorizer = CountVectorizer(
    min_df=20, 
    stop_words='english',
    token_pattern=three_words_pattern) 
vectorizer.fit(newsgroup_data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=20,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b\\w{3,}\\b', tokenizer=None,
        vocabulary=None)

<font color = green >

#### Review feratures 

</font>



In [42]:
print ('len of features = {:,}\n'.format(len(vectorizer.get_feature_names())))
print (vectorizer.get_feature_names()[:40])


len of features = 902

['000', '100', '1990', '1992', '1993', '200', '2nd', '300', '400', '486', '500', '800', 'ability', 'able', 'accept', 'accepted', 'access', 'according', 'actual', 'actually', 'add', 'addition', 'additional', 'address', 'advance', 'advice', 'age', 'ago', 'agree', 'ahead', 'air', 'allow', 'alt', 'america', 'american', 'answer', 'answers', 'anybody', 'apparently', 'appears']


<font color = green >

#### Vectorize data set

</font>



In [43]:
newsgroup_data_vectorized= vectorizer.transform(newsgroup_data)
print (newsgroup_data_vectorized)

  (0, 23)	1
  (0, 33)	1
  (0, 58)	1
  (0, 76)	1
  (0, 326)	1
  (0, 335)	1
  (0, 386)	1
  (0, 409)	1
  (0, 451)	1
  (0, 456)	1
  (0, 515)	1
  (0, 529)	1
  (0, 545)	1
  (0, 727)	1
  (0, 843)	1
  (0, 900)	1
  (1, 33)	1
  (1, 34)	1
  (1, 84)	1
  (1, 184)	1
  (1, 201)	1
  (1, 214)	1
  (1, 231)	2
  (1, 241)	1
  (1, 324)	1
  :	:
  (1998, 622)	1
  (1998, 625)	3
  (1998, 688)	1
  (1998, 698)	2
  (1998, 726)	1
  (1998, 804)	1
  (1998, 805)	1
  (1998, 810)	10
  (1998, 813)	2
  (1998, 814)	1
  (1998, 816)	1
  (1998, 818)	1
  (1998, 844)	1
  (1998, 882)	2
  (1998, 899)	1
  (1999, 171)	1
  (1999, 194)	1
  (1999, 205)	1
  (1999, 213)	1
  (1999, 276)	2
  (1999, 308)	1
  (1999, 344)	1
  (1999, 669)	1
  (1999, 832)	1
  (1999, 874)	1


<font color = green >

#### Create gensim corpus

</font>



In [44]:
corpus = gensim.matutils.Sparse2Corpus(newsgroup_data_vectorized, documents_columns=False)
# comparing to using corpora.Dictionary:
# corpus = [dictionary.doc2bow(text) for text in texts] 
[item for item in corpus][:5]


[[(23, 1),
  (33, 1),
  (58, 1),
  (76, 1),
  (326, 1),
  (335, 1),
  (386, 1),
  (409, 1),
  (451, 1),
  (456, 1),
  (515, 1),
  (529, 1),
  (545, 1),
  (727, 1),
  (843, 1),
  (900, 1)],
 [(33, 1),
  (34, 1),
  (84, 1),
  (184, 1),
  (201, 1),
  (214, 1),
  (231, 2),
  (241, 1),
  (324, 1),
  (332, 1),
  (359, 1),
  (363, 1),
  (365, 1),
  (409, 1),
  (430, 3),
  (451, 1),
  (475, 1),
  (492, 2),
  (525, 2),
  (605, 1),
  (633, 2),
  (642, 1),
  (674, 1),
  (688, 1),
  (709, 1),
  (750, 1),
  (777, 1),
  (823, 1),
  (838, 1),
  (874, 1),
  (896, 1)],
 [(25, 1),
  (26, 1),
  (63, 1),
  (120, 1),
  (231, 1),
  (297, 1),
  (326, 1),
  (344, 1),
  (373, 1),
  (423, 1),
  (442, 1),
  (444, 1),
  (448, 2),
  (465, 1),
  (572, 1),
  (653, 1),
  (659, 1),
  (714, 1),
  (777, 1),
  (779, 1),
  (781, 1),
  (818, 1),
  (836, 1),
  (855, 1),
  (890, 1),
  (898, 1)],
 [(4, 1),
  (17, 2),
  (18, 1),
  (22, 1),
  (42, 1),
  (48, 2),
  (68, 1),
  (78, 1),
  (86, 1),
  (94, 1),
  (117, 1),
  (119, 1)

<font color = green >

#### Create id2word dictionary

</font>



In [45]:
id_map = dict((v, k) for k, v in vectorizer.vocabulary_.items()) 

<font color = green >

#### Generate LDA model

</font>



In [46]:
ldamodel = gensim.models.ldamodel.LdaModel (corpus, num_topics=6, id2word=id_map, passes=25, random_state=34)
# Comparing to corpora.Dictionary
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary, passes=20, random_state= 0)


<font color = green >

#### Review topics

</font>



In [47]:
ldamodel.print_topics(num_topics=6,num_words=10)

[(0,
  '0.025*"edu" + 0.019*"com" + 0.018*"use" + 0.018*"thanks" + 0.016*"does" + 0.015*"know" + 0.011*"mail" + 0.010*"apple" + 0.009*"help" + 0.008*"want"'),
 (1,
  '0.061*"drive" + 0.039*"disk" + 0.030*"scsi" + 0.027*"drives" + 0.027*"hard" + 0.025*"controller" + 0.021*"card" + 0.018*"rom" + 0.016*"cable" + 0.016*"floppy"'),
 (2,
  '0.024*"people" + 0.022*"god" + 0.012*"atheism" + 0.012*"think" + 0.012*"believe" + 0.012*"don" + 0.010*"does" + 0.010*"just" + 0.009*"argument" + 0.009*"say"'),
 (3,
  '0.023*"game" + 0.021*"year" + 0.020*"team" + 0.013*"games" + 0.013*"play" + 0.011*"good" + 0.011*"don" + 0.010*"think" + 0.010*"season" + 0.010*"players"'),
 (4,
  '0.035*"space" + 0.019*"nasa" + 0.018*"data" + 0.013*"information" + 0.013*"available" + 0.013*"center" + 0.011*"ground" + 0.010*"research" + 0.010*"000" + 0.010*"new"'),
 (5,
  '0.017*"just" + 0.017*"like" + 0.016*"don" + 0.012*"car" + 0.012*"time" + 0.011*"think" + 0.011*"good" + 0.010*"know" + 0.008*"way" + 0.008*"people"')]

<font color = green >

#### Name topics

</font>



In [48]:
topics_names= ['Education', 'Computers & IT', 'Religion', 'Sports', 'Science','Society & Lifestyle']

<font color = green >

#### Classify the new text 

</font>



In [53]:
import numpy as np

In [54]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "] 


In [55]:
doc_vectorized= vectorizer.transform(new_doc) # input param is list
new_doc_corpus = gensim.matutils.Sparse2Corpus(doc_vectorized, documents_columns=False)
doc_topics = ldamodel.get_document_topics(new_doc_corpus)
list(doc_topics)

[[(0, 0.033417623),
  (1, 0.033340912),
  (2, 0.03351664),
  (3, 0.03380149),
  (4, 0.83228344),
  (5, 0.03363986)]]

In [56]:
def elicit_topic_name(doc_topics):    
    return topics_names[np.squeeze(np.array(doc_topics))[:,1].argmax()]
elicit_topic_name(doc_topics)

'Science'

<font color = green >

## Home Task 

</font>


<font color = green >

### Topic Modeling 

</font>

[voted-kaggle-dataset](https://www.kaggle.com/canggih/voted-kaggle-dataset/version/2#voted-kaggle-dataset.csv)

In [18]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from nltk.corpus import stopwords
import pandas as pd
import gensim

fn=  os.path.join(path , 'voted-kaggle-dataset.csv')
df = pd.read_csv(fn)

In [17]:
print ('len of texts= {:,}'.format(len(df)))
index = 10 
df.loc[index, 'Description']

len of texts= 2,150


'These files contain complete loan data for all loans issued through the 2007-2015, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. The file containing loan data through the "present" contains complete loan data for all loans issued through the previous completed calendar quarter. Additional features include credit scores, number of finance inquiries, address including zip codes, and state, and collections among others. The file is a matrix of about 890 thousand observations and 75 variables. A data dictionary is provided in a separate file. k'

In [29]:
df_data_set = df["Description"]

In [30]:
en_stop  = set(stopwords.words('english'))
p_stemmer = PorterStemmer()

def tokenize(df_data_set):
    texts = []
    for doc in df_data_set:
        # tokenize document string
        doc = str(doc)
        raw = doc.lower()
        tokens = word_tokenize(raw)

        # remove stop words from tokens
        tokens = [token for token in tokens if token not in en_stop]

        # stem tokens
        tokens = [p_stemmer.stem(token) for token in tokens]

        # add tokens to list
        texts.append(tokens)
    return texts

texts = tokenize(df_data_set)
texts[0]

['dataset',
 'contain',
 'transact',
 'made',
 'credit',
 'card',
 'septemb',
 '2013',
 'european',
 'cardhold',
 '.',
 'dataset',
 'present',
 'transact',
 'occur',
 'two',
 'day',
 ',',
 '492',
 'fraud',
 '284,807',
 'transact',
 '.',
 'dataset',
 'highli',
 'unbalanc',
 ',',
 'posit',
 'class',
 '(',
 'fraud',
 ')',
 'account',
 '0.172',
 '%',
 'transact',
 '.',
 'contain',
 'numer',
 'input',
 'variabl',
 'result',
 'pca',
 'transform',
 '.',
 'unfortun',
 ',',
 'due',
 'confidenti',
 'issu',
 ',',
 'provid',
 'origin',
 'featur',
 'background',
 'inform',
 'data',
 '.',
 'featur',
 'v1',
 ',',
 'v2',
 ',',
 '...',
 'v28',
 'princip',
 'compon',
 'obtain',
 'pca',
 ',',
 'featur',
 'transform',
 'pca',
 "'time",
 "'",
 "'amount",
 "'",
 '.',
 'featur',
 "'time",
 "'",
 'contain',
 'second',
 'elaps',
 'transact',
 'first',
 'transact',
 'dataset',
 '.',
 'featur',
 "'amount",
 "'",
 'transact',
 'amount',
 ',',
 'featur',
 'use',
 'example-depend',
 'cost-senst',
 'learn',
 '.',
 '

In [31]:
dictionary = corpora.Dictionary(texts) # this is alternative way - without using count vectorizer
print (type(dictionary), dictionary)
for k,w in dictionary.items():
    print (k,w)

<class 'gensim.corpora.dictionary.Dictionary'> Dictionary(35583 unique tokens: ['%', "'", "'amount", "'class", "'time"]...)
0 %
1 '
2 'amount
3 'class
4 'time
5 (
6 )
7 ,
8 .
9 ...
10 //mlg.ulb.ac.b
11 //mlg.ulb.ac.be/artml
12 //mlg.ulb.ac.be/bruf
13 0
14 0.172
15 1
16 2013
17 2015
18 284,807
19 492
20 :
21 a.
22 account
23 accuraci
24 amount
25 analys
26 andrea
27 area
28 auprc
29 avail
30 background
31 big
32 bontempi
33 bruxel
34 caelen
35 calibr
36 card
37 cardhold
38 case
39 cidm
40 cite
41 class
42 classif
43 collabor
44 collect
45 compon
46 comput
47 confidenti
48 confus
49 contain
50 cost-senst
51 credit
52 current
53 curv
54 dal
55 data
56 dataset
57 day
58 de
59 detail
60 detect
61 due
62 elaps
63 european
64 example-depend
65 featur
66 first
67 fraud
68 gianluca
69 given
70 group
71 highli
72 http
73 ieee
74 imbal
75 inform
76 input
77 intellig
78 issu
79 johnson
80 learn
81 libr
82 machin
83 made
84 matrix
85 meaning
86 measur
87 mine
88 numer
89 obtain
90 occur
91 olivi
92

833 send
834 serv
835 sha-256
836 share
837 soon
838 spare
839 student
840 technic
841 thank
842 thu
843 timestamp
844 togeth
845 trade
846 trust
847 unforseen
848 unintuit
849 unix
850 variou
851 via
852 viewer
853 volum
854 wait
855 way
856 weight
857 -freeformresponses.csv
858 1000
859 14
860 16,000
861 16,716
862 16.4
863 171
864 25th
865 5
866 50
867 5th
868 7th
869 allow
870 attempt
871 august
872 break
873 calcul
874 channel
875 choic
876 column-wis
877 compens
878 comprehens
879 conduct
880 convers
881 conversionrates.csv
882 correspond
883 day-to-day
884 decod
885 describ
886 edg
887 email
888 employ
889 establish
890 exclud
891 flag
892 folk
893 form
894 free
895 freeform
896 freeformresponses.csv
897 happen
898 ident
899 industry-wid
900 initi
901 interact
902 kind
903 less
904 live
905 median
906 month
907 multipl
908 multiplechoiceresponses.csv
909 non-random
910 novemb
911 open-end
912 option
913 packag
914 primarili
915 proceed
916 protect
917 pud
918 quantmod
919 r
920 

1583 n
1584 needl
1585 none
1586 nuclei
1587 nucleu
1588 o.
1589 optim
1590 p.
1591 perimet
1592 perimeter^2
1593 portion
1594 radiu
1595 real-valu
1596 recod
1597 robust
1598 se
1599 server
1600 signific
1601 smooth
1602 softwar
1603 standard
1604 symmetri
1605 textur
1606 uw
1607 'reviews'
1608 1999
1609 2012.
1610 256,059
1611 260
1612 568,454
1613 74,258
1614 amateur
1615 amazon
1616 connoisseur
1617 evolut
1618 expertis
1619 j.
1620 leskovec
1621 mcauley
1622 octob
1623 onlin
1624 plain
1625 pull
1626 queri
1627 quick
1628 reviews.csv
1629 span
1630 www
1631 ~500,000
1632 +
1633 1.85
1634 155
1635 2.1
1636 2.2
1637 2013-2016
1638 2014-
1639 2014-2016
1640 20th
1641 absenc
1642 add
1643 alway
1644 assess
1645 bar
1646 benchmark
1647 cantril
1648 capita
1649 celebr
1650 character
1651 civil
1652 contrast
1653 corrupt
1654 dystopia
1655 effect
1656 equal
1657 equat
1658 exceed
1659 experi
1660 expert
1661 explain
1662 extent
1663 factor
1664 favor
1665 fourth
1666 freedom
1667 gain
1

2332 vari
2333 8600
2334 axi
2335 behaviour
2336 brigg
2337 carl
2338 categoris
2339 clung
2340 code/typ
2341 cognit
2342 distinct
2343 extrovers
2344 googl
2345 intp
2346 introvers
2347 intuit
2348 judg
2349 jung
2350 jungian
2351 later
2352 letter
2353 mbti
2354 mind
2355 myer
2356 personalitycaf
2357 pipe
2358 reveal
2359 scientif
2360 sens
2361 short
2362 suggest
2363 surround
2364 thought
2365 typolog
2366 unreli
2367 use/valid
2368 written
2369 |||
2370 2003
2371 academ
2372 alma
2373 arabia
2374 barro-le
2375 china
2376 compris
2377 controversi
2378 critic
2379 disagre
2380 educ
2381 expenditur
2382 fair
2383 focus
2384 gross
2385 human
2386 influenti
2387 institut
2388 instruct
2389 kingdom
2390 levi
2391 mater
2392 non-english-instruct
2393 practic
2394 privat
2395 saudi
2396 shanghai
2397 supplementari
2398 undermin
2399 unesco
2400 abus
2401 advantag
2402 analyz
2403 attitud
2404 bring
2405 care
2406 care_opt
2407 choos
2408 company/organ
2409 consequ
2410 cowork
2411 disord

3082 flaw
3083 forward
3084 genet
3085 harvest
3086 highest-r
3087 ideal
3088 impress
3089 intens
3090 linger
3091 manhattan
3092 melt
3093 narrowli
3094 ordinari
3095 passabl
3096 pound
3097 praiseworthi
3098 premium
3099 profici
3100 profil
3101 promin
3102 puriti
3103 resid
3104 satisfactori
3105 solid
3106 storag
3107 straight
3108 summar
3109 superior
3110 terroir
3111 transcend
3112 unpalat
3113 vision
3114 14.3
3115 2014.
3116 329
3117 4.5
3118 affiliated_base_num
3119 american_b01362.csv
3120 b02512
3121 b02598
3122 b02617
3123 b02682
3124 b02764
3125 b02765
3126 b02835
3127 b02836
3128 borough
3129 carmel_b00256.csv
3130 coarse-grain
3131 commiss
3132 danach-ni
3133 date/tim
3134 dial7_b00887.csv
3135 diplo_b01196.csv
3136 dispatch
3137 dispatching_base_num
3138 dreist
3139 drinnen
3140 driver
3141 essenti
3142 federal_02216.csv
3143 fhv
3144 fine-grain
3145 firstclass_b01536.csv
3146 fivethirtyeight
3147 flv
3148 foil
3149 for-hir
3150 grun
3151 hfv
3152 highclass_b01717.csv


...

31823 soar
31824 sobrado
31825 sofiya
31826 sohaib
31827 sohaibomar
31828 soham
31829 sohel
31830 sohier
31831 sohinibhattacharya
31832 sol
31833 solanki
31834 soldo
31835 soler-compani
31836 somasundaram
31837 somat
31838 somavarapu
31839 somesh
31840 sommenoob
31841 somnath
31842 sonali
31843 sonamsrivastava
31844 sonavan
31845 sonawan
31846 songsong
31847 sonneland
31848 soo
31849 soojung
31850 soorajm
31851 soroosh
31852 sotopia
31853 souci
31854 soufian
31855 soufianeorama
31856 souhaiel
31857 souhail
31858 soukaina
31859 souliman
31860 souman
31861 soumitra
31862 sourabhmitt
31863 sourav
31864 souravmaharana
31865 sousa
31866 sovboc2018
31867 sowhit
31868 sowmiya
31869 soywu
31870 spadafora
31871 spadim
31872 spencer
31873 sprabakar
31874 sreeram
31875 sreyansh
31876 sridhar
31877 sridharan
31878 srigiriraju
31879 srihari
31880 sriharirao
31881 srikanth
31882 srilakshmi
31883 srilakshminandamuri
31884 srilbg
31885 srinath
31886 srinivasan
31887 srinivasrao
31888 srinivinnakota
31

32572 æžç
32573 æž—æ¹§æ£®
32574 ç
32575 ç§‹ä¹‹çµç¾½
32576 éœœé›ªåƒå¹´
32577 ê²½ë¦¼
32578 ê³½
32579 ë¯¼
32580 ë²¤ìž
32581 î¹î´î·ï‚
32582 îœî±ïî¹î¿ï‚
32583 îœî¹ï‡î±î·î
32584 ðð
32585 ðð½ð´ñ€ðµð¹ð¢ð¸ð¼ð¾ñ
32586 ðñ€ñ‚ðµð¼
32587 ð´ñ‹ð·ð¶ð°ð½ð¡ð°ð³ð¸ð¼ð±ð°ðµð²
32588 ðµðµð²
32589 ðµðºñð°ð½ð´ñ€
32590 ð¸ñð°
32591 ðÿñƒð³ð°ñ‡ðµð²ð°
32592 ðœð°ðºññžñ‚ðµð½ðºð¾
32593 ðœð¸ñ…ð°ð¸ð
32594 ð–ñƒð
32595 ð›ñð½
32596 ø§ø¨
32597 ø·ùšùø£ø­ù…ø¯øºù
32598 ø¹ø¨ø¯ø§ù
32599 ù
32600 „
32601 ‡m
32602 ‹ch
32603 ‹å³°
32604 ™i
32605 'hands-on
32606 //github.com/ageron/handson-ml/tree/master/datasets/h
32607 291-297.
32608 33.3
32609 aurélien
32610 géron
32611 housing_median_ag
32612 luí
32613 median_house_valu
32614 median_incom
32615 ocean_proxim
32616 rudimentari
32617 torgo
32618 total_bedroom
32619 total_room
32620 toyish
32621 10year
32622 added/consid
32623 data.go.kr
32624 buildings/structur
32625 dicsplin
32626 browser_us
32627 device_us
32628 is_respons
32629 323
32630 cant
32631 ml/dp
32632 motherboa

33322 22:00u
33323 2°
33324 365day
33325 3week
33326 4933
33327 60225
33328 66000
33329 9:00am
33330 9:00u-22:00
33331 black-point
33332 blackpoint
33333 collid
33334 day..
33335 day/night
33336 dayblock
33337 daytim
33338 feast
33339 fr-sa-so
33340 fr/so
33341 giss
33342 google-api
33343 guard-capit
33344 guard-pharmaci
33345 halv
33346 haversin
33347 kalend
33348 mid/week
33349 midweek
33350 midweek/weekend
33351 mo-tu-we-th
33352 mse
33353 night-guard
33354 nightblock
33355 nightshift
33356 pharmaci
33357 pharmacist
33358 sample..
33359 school/holiday
33360 sinksen
33361 sun/moon
33362 superimpos
33363 weekend/midweek
33364 weekendblock
33365 'flat
33366 2011-2016
33367 analytics/forecasting/consolid
33368 worldsteel
33369 www.worldsteel.org
33370 htsa
33371 merchandis
33372 usitc
33373 3a
33374 80-20
33375 benwing.com
33376 delozi
33377 document-geoloc
33378 geoannot
33379 geojson
33380 grantdelozi
33381 polygon-centroid
33382 rebellion
33383 size-2
33384 stand-off
33385 training/t

34072 shoton
34073 shoton_detail.csv
34074 0.0004
34075 0.0005
34076 0.0200
34077 39.9
34078 3seri
34079 co_tailpip
34080 g/
34081 hc_tailpip
34082 nox_tailpip
34083 sedan
34084 tailpip
34085 these
34086 vehicle_spe
34087 𝑎𝑛𝑑
34088 resumpt
34089 sona
34090 //www.imcb.osaka-u.ac.jp/nakai/psort.html
34091 109-115.
34092 11:95-110
34093 14:897-911
34094 336
34095 alm1
34096 alm2
34097 alom
34098 amino
34099 chg
34100 cleavabl
34101 cs.berkeley.edu
34102 cytoplasm
34103 e.coli
34104 gram-neg
34105 gvh
34106 heijn
34107 hoc
34108 horton
34109 imcb.osaka-u.ac.jp
34110 kanehisa
34111 kenta
34112 lipoprotein
34113 mcg
34114 mcgeoch
34115 membran
34116 minoru
34117 n-terminu
34118 nakai
34119 om
34120 oml
34121 paulh
34122 peptidas
34123 periplasm
34124 perisplasm
34125 probablist
34126 suita
34127 swiss-prot
34128 sytem
34129 uncleav
34130 yamada-oka
34131 +-
34132 //archive.ics.uci.edu/ml/datasets/forest+fir
34133 //www.dsi.uminho.pt/~pcortez/fires.pdf
34134 0.00
34135 0.40
34136 10-fold
3413

35072 rumrat
35073 rumratings.com
35074 //www.census.gov/data/tables/1995/demo/educational-attainment/p20-489.html
35075 //www.census.gov/data/tables/2005/demo/educational-attainment/cps-detailed-tables.html
35076 //www.census.gov/data/tables/2015/demo/education-attainment/p20-578.html
35077 //www.census.gov/topics/education/educational-attainment/data/tables.all.html
35078 18-24
35079 18-24.
35080 18_24
35081 25-34
35082 29-34
35083 coarser
35084 bypass
35085 dentist
35086 misinform
35087 read_delim
35088 ~2012
35089 //www.fortnitechests.info/
35090 2.1.0
35091 crit
35092 fortnit
35093 mag
35094 projectil
35095 reload
35096 soumydev
35097 underutil
35098 //www.usnews.com
35099 acceptance-r
35100 act-avg
35101 adelphi
35102 akron
35103 albani
35104 alliant
35105 argosi
35106 ashland
35107 auburn
35108 augusta
35109 azusa
35110 baptist
35111 baylor
35112 benedictin
35113 biola
35114 bloomington
35115 bois
35116 brandei
35117 brigham
35118 brunswick
35119 canyon
35120 capella
35121 carbo

In [32]:
print ('\nconvert tokenized documents into a document-term matrix')
corpus = [dictionary.doc2bow(text) for text in texts] # id and count
for item in corpus:
    print (item)


convert tokenized documents into a document-term matrix
[(0, 1), (1, 5), (2, 2), (3, 1), (4, 2), (5, 5), (6, 5), (7, 13), (8, 14), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 4), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 3), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 3), (56, 5), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 7), (66, 1), (67, 4), (68, 1), (69, 1), (70, 1), (71, 1), (72, 3), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 3), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), 

[(5, 2), (6, 2), (7, 3), (8, 10), (20, 1), (55, 2), (56, 2), (60, 1), (75, 1), (92, 1), (125, 1), (133, 1), (136, 1), (151, 1), (158, 2), (164, 1), (182, 1), (192, 1), (230, 3), (249, 1), (254, 1), (292, 1), (307, 3), (314, 1), (328, 1), (331, 1), (337, 1), (355, 2), (403, 2), (409, 1), (420, 1), (575, 1), (576, 1), (577, 1), (586, 1), (654, 1), (691, 1), (694, 1), (704, 1), (719, 1), (890, 3), (1059, 2), (1248, 1), (1325, 1), (1662, 1), (1741, 1), (1816, 2), (1819, 1), (1946, 1), (2035, 1), (2255, 2), (3197, 1), (3369, 2), (3410, 2), (3680, 1), (5424, 2), (5470, 1), (5899, 1), (6016, 1), (6682, 2), (6701, 1), (7913, 1), (8447, 1), (9318, 2), (10233, 1), (10694, 1), (12135, 1), (12917, 2), (17267, 1), (17269, 1), (17588, 1), (19122, 1), (21699, 1), (23226, 1), (23575, 3), (25154, 2), (25162, 1), (25163, 2), (25164, 1), (25165, 1), (25166, 1), (25167, 1), (25168, 1), (25169, 1), (25170, 1), (25171, 1), (25172, 1), (25173, 1), (25174, 8), (25175, 1)]
[(5, 1), (6, 1), (7, 4), (8, 4), (9, 

In [37]:
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=2, id2word=dictionary, passes=20, random_state= 0)

In [38]:
ldamodel.print_topics(num_topics=5,num_words=10)

[(0,
  '0.057*"," + 0.050*"." + 0.027*":" + 0.022*")" + 0.021*"(" + 0.014*"data" + 0.010*"dataset" + 0.007*"use" + 0.005*"?" + 0.005*"\'\'"'),
 (1,
  '0.048*"univers" + 0.032*"." + 0.015*"dataset" + 0.012*"-" + 0.012*"," + 0.009*"state" + 0.009*"--" + 0.007*"descript" + 0.006*"data" + 0.006*"yet"')]

<font color = green >

## Learn more
</font>

Latent Dirichlet allocation
<br>
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation


<font color = green >

## Next lesson: Clustering 
</font>

